In [81]:
import pandas as pd
import json
import re

In [82]:
with open('/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/documents/process/qaset_rd.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.json_normalize(data)

In [83]:
def process_references(references):
    if not references:
        return references
    processed_refs = []
    for ref in references:
        # Tìm vị trí của từ "khoản" đi kèm với số
        match = re.search(r'khoản \d+', ref)
        if match:
            index = match.start()
        else:
            # Nếu không tìm thấy "khoản" thì tìm "Điều"
            index = ref.find("Điều")
        if index != -1:
            processed_refs.append(ref[index:])
        else:
            processed_refs.append(ref)  # Giữ nguyên nếu không tìm thấy "khoản" hoặc "Điều"
    return processed_refs

In [84]:
def split_references(references):
    if not references:
        return None
    split_refs = []
    for ref in references:
        # Tìm các phần "khoản" và "Điều"
        match_khoan = re.search(r'khoản \d+', ref)
        match_dieu = re.search(r'Điều \d+', ref)
        if match_khoan and match_dieu:
            part_khoan = match_khoan.group(0)
            part_dieu = match_dieu.group(0)
            # Tạo chuỗi "khoản ... Điều ..."
            part = f"{part_khoan} {part_dieu}"
            # Tìm phần còn lại của luật hoặc nghị định
            law = ref.replace(part_khoan, '').replace(part_dieu, '').strip(', ').strip()
            split_refs.append([part, law])
        elif match_dieu:
            part_dieu = match_dieu.group(0)
            law = ref.replace(part_dieu, '').strip(', ').strip()
            split_refs.append([part_dieu, law])
    return split_refs if split_refs else None

In [85]:
df['references'] = df['references'].apply(process_references)
df['references'] = df['references'].apply(split_references)

In [86]:
df = df[df['references'].map(lambda x: x != [])]
df = df[df['references'].notna()]
df.dropna(axis=1, how='all', inplace=True)

In [87]:
df.head()

,question,answer,references
0,Hội đồng giải thể doanh nghiệp do Nhà nước nắm...,Theo quy định tại Điều 44 Nghị định 23/2022/NĐ...,"[[Điều 44, Nghị định 23/2022/NĐ-CP]]"
1,Có được quyền kéo dài thêm thời hạn giải thể d...,Theo quy định tại Điều 47 Nghị định 23/2022/NĐ...,"[[Điều 47, Nghị định 23/2022/NĐ-CP]]"
2,Khi giải thể doanh nghiệp do Nhà nước nắm giữ ...,Theo quy định tại khoản 2 Điều 39 Nghị định 23...,"[[khoản 2 Điều 39, Nghị định 23/2022/NĐ-CP]]"
3,Hồ sơ giải thể doanh nghiệp mới nhất năm 2023 ...,Căn cứ theo quy định tại khoản 1 Điều 210 Luật...,"[[khoản 1 Điều 210, Luật Doanh nghiệp 2020]]"
4,Thứ tự thanh toán khoản nợ của doanh nghiệp gi...,Theo quy định tại khoản 5 Điều 208 Luật Doanh ...,"[[khoản 5 Điều 208, Luật Doanh nghiệp 2020]]"


In [88]:
with open('/Users/nhotin/Documents/GitHub/LegalBizAI_project/data/documents/process/clause/qaset_clause.json', 'w', encoding='utf-8') as file:
    json.dump(df.to_dict(orient='records'), file, ensure_ascii=False, indent=4)